In [5]:
import pandas as pd

In [6]:
# my index column or the first column is unnamed
df = pd.read_csv("./data/progressive_moves_500.csv", index_col = 0)
df.head()

,opening_name,prev_moves,last_move
0,Slav Defense: Exchange Variation,NaN,d4
1,Slav Defense: Exchange Variation,d4,d5
2,Slav Defense: Exchange Variation,d4 d5,c4
3,Slav Defense: Exchange Variation,d4 d5 c4,c6
4,Slav Defense: Exchange Variation,d4 d5 c4 c6,cxd5


# Loading API keys

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
openai_api_key = os.getenv("openai_api_key")